In [2]:
# package imported
import torch
import numpy as np
import json
import re
import torch.nn as nn

# file path
file_path="E:\\AAA\\nlp\\AA_my_poem_generator\\songci\\data.json"
voc_path="E:\\AAA\\nlp\\AA_my_poem_generator\\voc\\"
model_path="E:\\AAA\\nlp\\AA_my_poem_generator\\checkpoints_sc\\sc"
pz_path="E:\\AAA\\nlp\\AA_my_poem_generator\\平仄\\"

#poem selecting
min_len=10
max_len=200


# embedding setting
word2id = {}  # 字典转换，汉字转化为相应的index
id2word = {}  # index转换回汉字

# training setting
lr = 0.001
epoch = 20  # 训练整套数据的次数
embedding_dim = 128  # 词向量的维度
hidden_dim = 256 # 隐层的维度
batch_size = 128  # 批处理的量

  

In [3]:
with open(voc_path+"word2id.json") as f0:
    word2id=json.load(f0)
with open(voc_path+"id2word.json") as f1:
    id2word=json.load(f1) 

In [4]:
data=torch.load('E:\\AAA\\nlp\\AA_my_poem_generator\\songci\\tensor_data.pt')

In [5]:
class POEM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(POEM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)  #one hot version to embedding dim
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, num_layers=2)
        self.linear1 = nn.Linear(self.hidden_dim, vocab_size)

    def forward(self, input, hidden=None):
        #向前遍历序列，产生一系列output，最后将output整合与target比较进行loss compute和参数调整
        seq_len, batch_size = input.size()
        if hidden is None:
            h = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
            c = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
        else:
            h, c = hidden
        # size: (seq_len,batch_size,embeding_dim)
        embeds = self.embeddings(input)
        # output size: (seq_len,batch_size,hidden_dim)
        output, hidden = self.lstm(embeds, (h, c))

        # size: (seq_len*batch_size,vocab_size)
        output = self.linear1(output.view(seq_len * batch_size, -1))
        return output, hidden


In [6]:
import torch
import random
max_sentence_len=30
#根据开头生成诗歌
def generate(model, start_words):
    results = list(start_words) #首先将结果置为list of start words
    start_word_len = len(start_words)
    input = torch.Tensor([word2id['S']]).view(1, 1).long() #第一个input为开始符号'S'
    hidden = None #传入的hidden是None，在模型内部初始化为0
    flag=False #判断是否为第一个句号
    rand=False #在逗号之后设置为随机生成
    for i in range(max_sentence_len):
        output, hidden = model(input, hidden)
        if i < start_word_len:
            w = results[i]
            input = input.data.new([word2id[w]]).view(1, 1)
        else:
            if rand==False: #如果不是在逗号后面直接选择LSTM输出的概率最大的汉字
                top_index = output.data[0].topk(1)[1][0].item()
            else:
                while top_index==word2id['，'] or top_index==word2id['。'] or top_index==word2id[' ']:
                    top_index = output.data[0].topk(10)[1][random.randint(1,10)-1].item()
                rand=False

            if top_index==word2id['。'] and flag==False:
                top_index=word2id['，']
                flag=True
            if top_index==word2id['，']:
                rand=True
            w = id2word[str(top_index)]
            results.append(w)
            input = input.data.new([top_index]).view(1, 1)
        if w == 'E':#如果下一个概率最大的是结束符号
            del results[-1]
            break
        ret_str=""
        for item in results:
            ret_str=ret_str+item
        ret_str=ret_str.replace(' ','')
    return ret_str

In [7]:
model = POEM(len(word2id), embedding_dim, hidden_dim)
model.load_state_dict(torch.load('E:\\AAA\\nlp\\AA_my_poem_generator\\16307130322_苏心怡_nlp期末pj\\checkpoints\\sc'+'_'+str(10)+'.pth'))

In [8]:
generate(model, "金玉盏")

'金玉盏，金钗金缕，玉佩金杯，不管春风里。'

In [9]:
generate(model, "风雨")

'风雨，雨声吹断，一片春风。'

In [10]:
generate(model, "子夜歌")

'子夜歌声，梦断云山，一片春风雨。'

In [11]:
generate(model, "金缕衣")

'金缕衣，千古人间老。'

In [12]:
generate(model, "张爱玲")

'张爱玲珑，翠袖香深处。'

In [13]:
generate(model, "苏心怡，")

'苏心怡，一任一枝春色，谁与春风。'

In [14]:
for c in "梁思成":
    print(generate(model, c))

梁岛，千骑千里，长是江南路。
思否，一枝春色，几度春风。
成璃苑，日日风流雨。


In [15]:
for c in "劝君莫惜金缕衣":
    print(generate(model, c))

劝酒，不妨人道，无计无情绪，且是人间，今夜归来，只恐花前醉。
君寿，千骑千里，一夜清明月。
莫奈匆匆
惜寿，十分明月明朝。
金萏缕，云鬟深处，依旧春风。
缕袖，绿窗红烛，翠袖香浓。
衣悴，不妨人道，只恐花前醉。


In [16]:
for c in "春江花月夜":
    print(generate(model, c))

春否，月明风月，又是一番春色。
江北北，何妨何处，又是一年春色，长是江南。
花蓉悴，又是人间，此事无人说。
月十二年时，月下人间有个人。
夜蓉怅望江南北，月明风月，又是一年春色，不堪回首。


In [18]:
for c in "张爱玲":
    print(generate(model, c))

张缈
爱恐匆匆
玲璃阙，十分明月，玉斧金杯。


In [123]:
for c in "金缕衣":
     print(generate(model, c, id2word, word2id))

金萏缕，满城风月，花下春风。
缕袖，春风吹雨，不见春风。
衣悴，人间何处，一片春风雨。


In [124]:
for c in "林徽因":
     print(generate(model, c, id2word, word2id))

林蓉岛，十分明月，月下人间。
徽酊子，千骑千里，一曲清明月，玉斧金杯，金鼎金杯，更有人间世
因道否，只有人间世。


In [11]:
a=torch.tensor([1,2,3,4,5,6])

In [122]:
a.topk(2)[1]

tensor([5, 4])

In [123]:
import random
random.randint(1,2)

2

In [124]:
a=torch.tensor([0,0,0,0])

In [125]:
a.topk(1)

(tensor([0]), tensor([3]))

In [44]:
def generate_dingzhen(model,start_words):
    s=start_words
    for i in range(0,3):
        
        st=generate(model,s)
        print(st)
        s=st[-3:-1]

In [63]:
generate_dingzhen(model,"林徽因")

林徽因道否，月明风月，有人无数。
无数，千点千年。
千年，万里千年月。


In [68]:
generate_dingzhen(model,"梁思成")

梁思成否，梦里无人问我，此意难忘。
难忘得，人间何处，相对一枝春色。
春色，更是人间世。


In [70]:
generate_dingzhen(model,"徐志摩")

徐志摩挲玉佩，香风细雨，绿鬓香浓，千里千年。
千年，月明风月，一夜清明。
清明月，夜阑风月，月下人间，明月明朝。


In [79]:
generate_dingzhen(model,"陆小曼")

陆小曼卮，一枝春色，又是一番春色。
春色，人间何处，千里千年。
千年，此事无人问。


In [33]:
#对应“平”声的汉字
with open(pz_path+'ping.txt','r',encoding='utf8') as  f1:
    f11 = f1.readlines()
s_ping=""
for s in f11:
    s=re.sub("\s","",s)
    s_ping+=s
print(s_ping)

﻿东同铜桐筒童僮瞳筒中衷忠虫冲终戎崇嵩菘弓躬宫融雄熊穹穷冯风枫丰充隆空公功工攻蒙濛笼聋珑洪红鸿虹丛翁聪骢鬃通蓬篷烘潼蒙胧䓖匆砻峒罿螽狨沣癃幪梦潀讧嵕豵涷曈鲖翀忡嵩肜芃酆麷釭饛雺瞢璁谼恫嵷逢蝀侗絧艟犝氃爞瀜窿悾曚朦罞懵咙昽豅庞艐膧同詷戙穜种盅鼨茙駥芎沨蘴汎珫倥玒冢髳艨襱洚稯鬷猣螉蝬酮绒渱冬农宗锺钟龙舂松衡容蓉庸封胸雍浓重从逢缝踪茸峰蜂锋烽蛩筇慵恭供琮悰淙侬松茏凶墉镛佣溶镕醲秾蛬邛共憧鄘颙喁邕壅痈饔纵龚枞賨脓淞忪彸憃冲瑢葑匈凶汹禺雍噰廱丰𫓩銎懵蚣蹖榕犎跫恟灉襛蝩桻咚彤褣瞛橦江杠矼釭扛厖尨哤駹窗枞𫓩邦缸降泷双艭庞逢腔撞幢桩淙洚橦茳娏憃嵕谾𪻐漎豇蛖垹梆跫悾韸支枝移为垂吹陂碑奇宜仪皮儿离施知驰池规危夷师姿迟龟眉悲之芝时诗棋旗辞词期基疑姬丝司葵医帷思滋持随痴维卮麋螭麾墀弥慈遗肌脂雌披嬉尸狸炊湄篱兹差疲茨卑亏蕤陲骑曦歧岐谁斯私窥欹熙欺疵赀笞羁彝髭颐资糜饥衰锥姨楣夔祇涯伊蓍追缁箕椎罴罳篪釐萎匙凘脾坻嶷治骊妫飔尸綦怡尼漪累匜牺饴而鸱推縻璃祁绥逵咿巇酏𫄨羲羸肢骐訾狮嗤毗咨堕萁其醨粢雎睢漓蠡噫骓馗菑辎褵邳锜胝緌鳍迤蛇陴淇蜊漦媸淄丽牦弥筛纚厮氏痍榱娭壝齍蓠轙脽蕲耏嫠貔比椑僖鸃贻祺葹嘻㧑鹂瓷鹚铍琦骴洏洟骙唲嵋怩欙駓熹孜台蚩罹裨虒魑荽纰椸倕丕琪僛耆惟猗剂絁羇伾荠黧偲潍提醾埘魌犛鲕蓰祗禧峗庳居糍鬐栀澌踦戏锤蚳畸鵻戣劘褫椅胹榰埤跜磁腄栘崥錍嗺郿暆痿酾桵离梩謻貤貾㛤佳簃锱陑虽蚑摫郫仔觺嘻寅鄑蓷鲥麒茈委鍉鸸秠蜞頯蘼軝剞桋襹摛棰崎嵫胔褷隋箄眵黐邿齝蜲𠯠爢蛦娸觯樆姼柅榯鼒翍觜錤缌厜鉹趍鸤蘪柌犪秜耛怌泜澬跠黟瓻恞峓鄈沩逶藣騩蘲踟踑諆圮瓵覗洢倭爔桸劙宦嵯祎诐玼桤觭徛椔蠯罢俾岯帔枇笓毗琵貔霉禔呢狔倪嫘樏藟梨犁蔾缡漓乖机赍伎埼只蜘砥隹郗弛篪锤槌蛳莳孖孳耔祠玭鹾罳撕濉宜宧诒迤眙崖嬴唯隗微薇晖辉辉徽挥翚韦围帏闱违霏菲妃騑绯飞非扉肥腓威祈旗畿机几讥矶鞿玑饥稀希晞衣依沂巍归祎诽淝痱欷豨楎餥厞蝛葳肵鐖叽鵗溰犩馡婓颀埼圻睎騩鱼渔初书舒居裾车渠蕖余予誉舆馀胥狙锄疏蔬梳虚嘘徐猪闾庐驴诸除储如墟垆菹琚旟玙与畬疽苴樗摅于箊茹蛆且沮袪祛蜍挐榈胪糈砠淤潴阹胠妤帤篨雎谞蘧腒鐻鶋椐纾袽躇橥趄璩鴽滁屠筡藘𦈌歔锄磲醵据瑹龉蠩唹驉摴蝑雓籧槠鵌呿魖藇铻疋咀蒘蒢湑衙涂狳洳薯虞愚娱隅刍无芜巫于盂臞衢儒濡襦须须株诛蛛殊铢瑜榆谀愉腴区驱躯朱珠趋扶符凫雏敷夫肤纡输枢厨俱驹模谟蒲胡湖瑚乎壶狐弧孤辜姑觚菰徒途涂荼图屠奴呼吾梧吴租卢鲈鑪芦苏酥乌枯粗都铺禺嵎诬竽雩吁

In [34]:
#对应“仄”声的汉字
with open(pz_path+'ze.txt','r',encoding='utf8') as  f2:
    f21 = f2.readlines()
s_ze=""
for s in f21:
    s=re.sub("\s","",s)
    s_ze+=s
print(s_ze)

﻿董动孔总笼澒汞桶蠓空嵷滃琫懵蓊拢唪洞挏蒙幪玤菶懂硐塕鬷埲侗唝翪肿种踵宠陇垄拥壅茸氄重冢奉覂勇涌踊甬俑蛹恐拱珙栱蛬巩竦悚耸汹湩拲溶恟駷鲖軵輁冗怂捧埇涌讲港棒蚌项缿玤傋耩纸只咫諟是轵枳砥抵氏靡彼毁毁委诡傀髓絫妓掎绮觜此泚豸褫徙屣蓰髀尔迩弭弥婢庳侈弛豕紫捶棰揣企旨指视美訾否兕几姊匕比妣轨水藟嚭唯止市恃征喜己纪跪技蚁迤酏俾鄙簋晷匦宄子梓矢洧鲔雉死履垒诔揆癸沚趾芷畤以已苡似耜汜姒巳祀史使驶耳珥駬里理里李俚鲤枲起芑杞屺跂士仕栜俟涘戺始峙痔齿矣拟薿耻祉滓笫胏垝嶲舣锜蒍薳玼廌玺逦酾纚鞞敉芊哆姼庀跬頍秕机氿欙圮痞痔儗坻褆嶬花阤旎址址悝娌嗺壝佹匜剞踦耔佌崺讄秭秠倚被底痏岿蕊尾鬼苇扆蚁卉虺几亹伟韪篚朏炜猪顗靴斐诽菲悱棐虮榧岂偯暐匪玮蜚蜰蘬唏语圉圄御龉敔吕侣旅膂纻苎宁杼伫羜与予渚煮汝茹暑鼠黍杵处贮褚楮醑糈谞湑女籹许拒距炬虡巨秬苣所楚础阻俎沮举莒筥叙序绪𫚈藇屿墅衙峿稆梠癙著稰巨駏岠鐻濋咀跙苴榉讵柜溆纾去儢麌雨羽禹宇舞父府鼓虎古股羖贾蛊土吐谱圃庾户树麈煦貐琥怙嵝蒟仵咻醹楰珇篓卤謱努弣罟肚妩沪龋枸斞冔邬鄅瞴蔖辅组乳弩补鲁橹橹竖腐卤数簿姥普拊每五庑斧聚午伍缕部柱矩武脯苦取抚浦主杜祖堵愈祜扈雇虏甫黼莆甒腑俯怃簠膴估诂盬牯瞽酤怒俣瑀祤喣踽窭楛稌浒诩栩寙炷拄剖鹉岵溥砮赌愈䉤伛偻蒌莽淦睹荠礼体米启醴陛洗邸底诋抵抵柢坻弟悌娣递涕济蠡澧欐鳢泚綮棨髀祢徯媞癠眯弥醍缇鲚泲挤氐抵砥泥昵睨奶溪蟹解骇买洒楷獬廌澥𫘤奶锴駴摆罢拐矮伙贿悔改采彩彩海在罪宰醢载喂铠恺待怠殆倍猥隗磈嵬嶵磥蕾癗儡礧櫑錞腇采绐诒蓓鼐颏骀欸琲垲廆浼頠汇瘣漼璀每亥乃轸敏允引尹尽忍准隼笋盾楯闵悯泯菌箘蚓靷纼诊眕畛胗紾哂肾脤膑牝冁赈窘蜃陨殒蠢蠢紧狁簨缜袗踳纯偆霣愍眹吮朕稹囷黾嶙吻粉蕴愤隐谨近恽忿槿菫坋弅坟听龀刎殷鼢阮远本晚苑返反阪损饭偃堰衮遁稳蹇幰巘楗揵婉菀蜿踠晼宛畹琬阃梱壸鲧悃捆辊绲鳟撙很恳垦畚圈盾刌绻鄢混沌鼹鰋蝘噂娩烜咺焜棍旱暖管琯满短馆缓盥款懒伞卵散伴诞浣瓒断笴侃算缵暵蜑但酂衎脘坦袒亶秆窾粄悍懑纂篹痯悹趱潸眼简版盏产限睅撰栈绾赧戁浐𡶴盏羼丳僝睆柬拣莞僩蝂眅钣輚馔皖板阪汕铲铣善遣浅典转衍犬选冕辇免展茧辩辨篆勉翦卷显饯践眄喘藓软巘蹇演岘栈舛荈扁脔谳阐兖娈跣腆鲜戬铉吮辫件笕琏蝡撚泫墠墡单畎褊惼艑瑑蜓殄腼甗蚬贙俛缅沔湎趼键狝襺黾蒇辗搴蜎琄𪾢愐洗齴鬋戭燹筅癣狷燀鄟諓钱趁僤韅毨隽揃歂缱涊嵃幝撰剸耎鞬谝匾撰宴姺碥俴缏萹餮沴捵晛篯啴𫗴膳鳝䏝僎稨楩娩謰沇馻卷蜒剪谫颤

In [35]:
id2pz={}
for item in word2id:
    if item in s_ping:
        id2pz[word2id[item]]="平"
    elif item in s_ze:
        id2pz[word2id[item]]="仄"
    else:
        if item!='，'and item!='。':
            id2pz[word2id[item]]="中"
print(id2pz)
    


{0: '仄', 1: '仄', 2: '仄', 3: '中', 4: '平', 5: '中', 6: '仄', 7: '仄', 8: '仄', 9: '仄', 10: '平', 11: '中', 12: '中', 13: '仄', 14: '平', 15: '平', 16: '平', 17: '中', 18: '仄', 19: '仄', 20: '平', 21: '仄', 22: '仄', 23: '仄', 24: '平', 25: '平', 26: '中', 27: '仄', 28: '平', 29: '平', 30: '平', 31: '仄', 32: '仄', 33: '平', 34: '仄', 35: '仄', 36: '仄', 37: '平', 38: '仄', 39: '仄', 40: '平', 41: '中', 42: '中', 43: '平', 44: '中', 45: '平', 46: '仄', 47: '平', 48: '平', 49: '平', 50: '平', 51: '平', 52: '平', 53: '仄', 54: '仄', 55: '中', 56: '仄', 57: '平', 58: '仄', 59: '平', 60: '中', 61: '仄', 62: '仄', 63: '仄', 64: '中', 65: '仄', 66: '仄', 67: '平', 68: '中', 69: '仄', 70: '平', 71: '仄', 72: '仄', 73: '平', 74: '仄', 75: '平', 76: '平', 77: '仄', 78: '仄', 79: '平', 80: '仄', 81: '仄', 82: '仄', 83: '仄', 84: '平', 85: '仄', 86: '平', 87: '仄', 88: '中', 89: '仄', 90: '平', 91: '仄', 92: '平', 93: '平', 94: '仄', 95: '仄', 96: '仄', 97: '仄', 98: '仄', 99: '平', 100: '平', 101: '仄', 102: '仄', 103: '平', 104: '仄', 105: '平', 106: '仄', 107: '仄', 108: '平', 109: '中', 110: '中',

In [36]:
with open(pz_path+"id2pz.json","w") as f:
    json.dump(id2pz,f)
with open(pz_path+"id2pz.json","r") as f:
    id2pz=json.load(f)


In [37]:
print(id2pz)

{'0': '仄', '1': '仄', '2': '仄', '3': '中', '4': '平', '5': '中', '6': '仄', '7': '仄', '8': '仄', '9': '仄', '10': '平', '11': '中', '12': '中', '13': '仄', '14': '平', '15': '平', '16': '平', '17': '中', '18': '仄', '19': '仄', '20': '平', '21': '仄', '22': '仄', '23': '仄', '24': '平', '25': '平', '26': '中', '27': '仄', '28': '平', '29': '平', '30': '平', '31': '仄', '32': '仄', '33': '平', '34': '仄', '35': '仄', '36': '仄', '37': '平', '38': '仄', '39': '仄', '40': '平', '41': '中', '42': '中', '43': '平', '44': '中', '45': '平', '46': '仄', '47': '平', '48': '平', '49': '平', '50': '平', '51': '平', '52': '平', '53': '仄', '54': '仄', '55': '中', '56': '仄', '57': '平', '58': '仄', '59': '平', '60': '中', '61': '仄', '62': '仄', '63': '仄', '64': '中', '65': '仄', '66': '仄', '67': '平', '68': '中', '69': '仄', '70': '平', '71': '仄', '72': '仄', '73': '平', '74': '仄', '75': '平', '76': '平', '77': '仄', '78': '仄', '79': '平', '80': '仄', '81': '仄', '82': '仄', '83': '仄', '84': '平', '85': '仄', '86': '平', '87': '仄', '88': '中', '89': '仄', '90': '平', '91': '仄

In [38]:
ping_id=[item for item in id2pz if id2pz[item]=="平"]
ping_id

['4',
 '10',
 '14',
 '15',
 '16',
 '20',
 '24',
 '25',
 '28',
 '29',
 '30',
 '33',
 '37',
 '40',
 '43',
 '45',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '57',
 '59',
 '67',
 '70',
 '73',
 '75',
 '76',
 '79',
 '84',
 '86',
 '90',
 '92',
 '93',
 '99',
 '100',
 '103',
 '105',
 '108',
 '114',
 '115',
 '121',
 '130',
 '134',
 '136',
 '139',
 '140',
 '141',
 '142',
 '144',
 '145',
 '148',
 '149',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '159',
 '160',
 '166',
 '167',
 '168',
 '175',
 '178',
 '180',
 '182',
 '186',
 '189',
 '190',
 '191',
 '192',
 '199',
 '201',
 '202',
 '206',
 '208',
 '210',
 '212',
 '215',
 '216',
 '217',
 '218',
 '221',
 '222',
 '223',
 '224',
 '225',
 '226',
 '229',
 '232',
 '233',
 '234',
 '236',
 '238',
 '240',
 '242',
 '243',
 '244',
 '245',
 '247',
 '248',
 '249',
 '252',
 '254',
 '257',
 '258',
 '259',
 '262',
 '264',
 '265',
 '267',
 '270',
 '272',
 '273',
 '274',
 '276',
 '277',
 '279',
 '282',
 '283',
 '285',
 '288',
 '289',
 '291',
 '294',
 '296',
 '297

In [39]:
ze_id=[item for item in id2pz if id2pz[item]=="仄"]
ze_id

['0',
 '1',
 '2',
 '6',
 '7',
 '8',
 '9',
 '13',
 '18',
 '19',
 '21',
 '22',
 '23',
 '27',
 '31',
 '32',
 '34',
 '35',
 '36',
 '38',
 '39',
 '46',
 '53',
 '54',
 '56',
 '58',
 '61',
 '62',
 '63',
 '65',
 '66',
 '69',
 '71',
 '72',
 '74',
 '77',
 '78',
 '80',
 '81',
 '82',
 '83',
 '85',
 '87',
 '89',
 '91',
 '94',
 '95',
 '96',
 '97',
 '98',
 '101',
 '102',
 '104',
 '106',
 '107',
 '111',
 '112',
 '113',
 '117',
 '118',
 '119',
 '120',
 '123',
 '124',
 '126',
 '127',
 '128',
 '129',
 '131',
 '132',
 '133',
 '135',
 '137',
 '138',
 '143',
 '146',
 '147',
 '156',
 '157',
 '161',
 '164',
 '165',
 '169',
 '170',
 '171',
 '172',
 '173',
 '174',
 '176',
 '177',
 '179',
 '181',
 '184',
 '185',
 '187',
 '188',
 '193',
 '196',
 '197',
 '198',
 '200',
 '203',
 '204',
 '205',
 '211',
 '213',
 '214',
 '219',
 '220',
 '231',
 '239',
 '246',
 '251',
 '253',
 '255',
 '256',
 '260',
 '261',
 '263',
 '266',
 '268',
 '269',
 '271',
 '275',
 '278',
 '281',
 '284',
 '286',
 '287',
 '290',
 '292',
 '293',
 

In [40]:
zhong_id=[item for item in id2pz if id2pz[item]=="中" if item!=word2id['，'] and item!=word2id['。']]
zhong_id

['3',
 '5',
 '11',
 '12',
 '17',
 '26',
 '41',
 '42',
 '44',
 '55',
 '60',
 '64',
 '68',
 '88',
 '109',
 '110',
 '116',
 '122',
 '125',
 '158',
 '162',
 '163',
 '183',
 '194',
 '195',
 '207',
 '209',
 '227',
 '228',
 '230',
 '235',
 '237',
 '241',
 '250',
 '280',
 '295',
 '304',
 '305',
 '317',
 '329',
 '338',
 '346',
 '357',
 '365',
 '374',
 '414',
 '415',
 '428',
 '429',
 '430',
 '443',
 '446',
 '448',
 '454',
 '469',
 '490',
 '514',
 '534',
 '557',
 '577',
 '597',
 '620',
 '635',
 '638',
 '646',
 '647',
 '656',
 '664',
 '668',
 '677',
 '678',
 '686',
 '690',
 '691',
 '693',
 '694',
 '697',
 '724',
 '748',
 '755',
 '761',
 '795',
 '824',
 '830',
 '844',
 '846',
 '867',
 '870',
 '885',
 '906',
 '910',
 '911',
 '919',
 '920',
 '937',
 '946',
 '957',
 '998',
 '999',
 '1005',
 '1006',
 '1013',
 '1022',
 '1024',
 '1049',
 '1099',
 '1102',
 '1107',
 '1123',
 '1148',
 '1152',
 '1159',
 '1160',
 '1170',
 '1196',
 '1205',
 '1217',
 '1242',
 '1252',
 '1256',
 '1261',
 '1267',
 '1292',
 '1305',

In [85]:
#清平乐词牌的格式（不含韵）

SLZL=["平中仄，平平仄仄平。","平平仄，中仄仄平平。"]



HXS=["仄仄平平仄仄平，平平仄仄仄平平。","平平仄仄仄平平。","仄仄平平平仄仄，平平仄仄仄平平。","平平仄仄仄平平。"]
HXS


['仄仄平平仄仄平，平平仄仄仄平平。', '平平仄仄仄平平。', '仄仄平平平仄仄，平平仄仄仄平平。', '平平仄仄仄平平。']

In [42]:
from random import choice
def get_id(ind_tensor,pz, select=True):
    if pz=='，':
        return word2id['，']
    elif pz=='。':
        return word2id['。']
    if pz=='平':
        pz_id=ping_id
    elif pz=='仄':
        pz_id=ze_id
    else:
        pz_id=zhong_id         
    item_l=[]
    for item in ind_tensor[0]:
        if str(int(item)) in pz_id:
            item_l.append(int(item))
            if len(item_l)>=3:
                return choice(item_l)
    if(len(item_l)>=1):
        return choice(item_l)
    temp_ind=ind_tensor[random.randint(1, len(ind_tensor))-1][0].item()
    while(temp_ind==word2id["，"]or temp_ind==word2id["。"]):
        temp_ind=ind_tensor[random.randint(1, len(ind_tensor))-1][0].item()
    return temp_ind
     
def generate_cipai(model,cipai_list,select_num):
    result=[]
    for sentence in cipai_list:
        gen_sen=""
        input = torch.Tensor([word2id['S']]).view(1, 1).long() #第一个input为开始符号'S'
        hidden=None
        for i in range(len(sentence)):
            output, hidden = model(input, hidden) 
            top_index = get_id(output.topk(select_num)[1],sentence[i])
            if sentence[i]=="，" or sentence[i]=="。":
                gen_sen+=sentence[i]
            else:
                gen_sen+=id2word[str(top_index)]
            input=input = input.data.new([top_index]).view(1, 1)
        result.append(gen_sen)
    return result
         
    

In [96]:
def pres(CP_name,num):
    l=generate_cipai(model,CP_name,num)
    print(l)
    res=[]
    for sen in l:
        ss=""
        for c in sen:
            try:
                ss+=((id2pz[str(word2id[c])]))
            except:
                ss+=c
        res.append(ss)
    print("\n")
    for s0 in res:
        print(s0)

In [105]:
pres(SLZL,30)

['蓉挲鹤，千骑一笑清。', '琶璃岛，一曲一声千。']


平中仄，平平仄仄平。
平平仄，仄仄仄平平。


In [98]:
pres(HXS,50)

['萏露寒浓碧水中，春风未肯有人间。', '璃蓉岛路远深深。', '逦处中明年事好，不妨此意在谁知。', '蓉琶蝶语脉难忘。']


仄仄平平仄仄平，平平仄仄仄平平。
平平仄仄仄平平。
仄仄平平平仄仄，平平仄仄仄平平。
平平仄仄仄平平。


In [173]:
a=torch.tensor([1,2,3])

In [192]:
word2id['S']

5294

In [39]:
id2pz

{'0': '仄',
 '1': '仄',
 '2': '仄',
 '3': '中',
 '4': '平',
 '5': '中',
 '6': '仄',
 '7': '仄',
 '8': '仄',
 '9': '仄',
 '10': '平',
 '11': '中',
 '12': '中',
 '13': '仄',
 '14': '平',
 '15': '平',
 '16': '平',
 '17': '中',
 '18': '仄',
 '19': '仄',
 '20': '平',
 '21': '仄',
 '22': '仄',
 '23': '仄',
 '24': '平',
 '25': '平',
 '26': '中',
 '27': '仄',
 '28': '平',
 '29': '平',
 '30': '平',
 '31': '仄',
 '32': '仄',
 '33': '平',
 '34': '仄',
 '35': '仄',
 '36': '仄',
 '37': '平',
 '38': '仄',
 '39': '仄',
 '40': '平',
 '41': '中',
 '42': '中',
 '43': '平',
 '44': '中',
 '45': '平',
 '46': '仄',
 '47': '平',
 '48': '平',
 '49': '平',
 '50': '平',
 '51': '平',
 '52': '平',
 '53': '仄',
 '54': '仄',
 '55': '中',
 '56': '仄',
 '57': '平',
 '58': '仄',
 '59': '平',
 '60': '中',
 '61': '仄',
 '62': '仄',
 '63': '仄',
 '64': '中',
 '65': '仄',
 '66': '仄',
 '67': '平',
 '68': '中',
 '69': '仄',
 '70': '平',
 '71': '仄',
 '72': '仄',
 '73': '平',
 '74': '仄',
 '75': '平',
 '76': '平',
 '77': '仄',
 '78': '仄',
 '79': '平',
 '80': '仄',
 '81': '仄',
 '82': '仄',
 '83': '仄',
 '

In [45]:
',' in s_ping

False

In [24]:
word2id

{'坼': 0,
 '卉': 1,
 '堵': 2,
 '尫': 3,
 '如': 4,
 'ō': 5,
 '郡': 6,
 '匝': 7,
 '业': 8,
 '制': 9,
 '玲': 10,
 '厓': 11,
 '蓖': 12,
 '藿': 13,
 '砂': 14,
 '帘': 15,
 '旒': 16,
 '飓': 17,
 '玺': 18,
 '众': 19,
 '盈': 20,
 '钥': 21,
 '蛰': 22,
 '拐': 23,
 '枨': 24,
 '为': 25,
 '撒': 26,
 '鹊': 27,
 '鳞': 28,
 '男': 29,
 '垆': 30,
 '怪': 31,
 '溥': 32,
 '佗': 33,
 '叙': 34,
 '雁': 35,
 '荐': 36,
 '豺': 37,
 '独': 38,
 '沔': 39,
 '炮': 40,
 '彀': 41,
 '菸': 42,
 '清': 43,
 '俊': 44,
 '柴': 45,
 '奏': 46,
 '查': 47,
 '身': 48,
 '门': 49,
 '妫': 50,
 '围': 51,
 '仔': 52,
 '鼓': 53,
 '枳': 54,
 '１': 55,
 '翠': 56,
 '搂': 57,
 '美': 58,
 '菅': 59,
 '缤': 60,
 '坝': 61,
 '苟': 62,
 '阐': 63,
 '锝': 64,
 '灼': 65,
 '带': 66,
 '砻': 67,
 '箜': 68,
 '赴': 69,
 '推': 70,
 '倚': 71,
 '冶': 72,
 '伎': 73,
 '浅': 74,
 '台': 75,
 '窥': 76,
 '酌': 77,
 '毁': 78,
 '猫': 79,
 '缉': 80,
 '谷': 81,
 '莽': 82,
 '舍': 83,
 '裳': 84,
 '殆': 85,
 '怆': 86,
 '嫩': 87,
 '辁': 88,
 '凿': 89,
 '纭': 90,
 '竺': 91,
 '坟': 92,
 '潺': 93,
 '记': 94,
 '淑': 95,
 '挂': 96,
 '算': 97,
 '粤': 98,
 '蒙': 99,
 '鄱': 100,

In [26]:
id2pz

{'0': '仄',
 '1': '仄',
 '2': '仄',
 '3': '中',
 '4': '平',
 '5': '中',
 '6': '仄',
 '7': '仄',
 '8': '仄',
 '9': '仄',
 '10': '平',
 '11': '中',
 '12': '中',
 '13': '仄',
 '14': '平',
 '15': '平',
 '16': '平',
 '17': '中',
 '18': '仄',
 '19': '仄',
 '20': '平',
 '21': '仄',
 '22': '仄',
 '23': '仄',
 '24': '平',
 '25': '平',
 '26': '中',
 '27': '仄',
 '28': '平',
 '29': '平',
 '30': '平',
 '31': '仄',
 '32': '仄',
 '33': '平',
 '34': '仄',
 '35': '仄',
 '36': '仄',
 '37': '平',
 '38': '仄',
 '39': '仄',
 '40': '平',
 '41': '中',
 '42': '中',
 '43': '平',
 '44': '中',
 '45': '平',
 '46': '仄',
 '47': '平',
 '48': '平',
 '49': '平',
 '50': '平',
 '51': '平',
 '52': '平',
 '53': '仄',
 '54': '仄',
 '55': '中',
 '56': '仄',
 '57': '平',
 '58': '仄',
 '59': '平',
 '60': '中',
 '61': '仄',
 '62': '仄',
 '63': '仄',
 '64': '中',
 '65': '仄',
 '66': '仄',
 '67': '平',
 '68': '中',
 '69': '仄',
 '70': '平',
 '71': '仄',
 '72': '仄',
 '73': '平',
 '74': '仄',
 '75': '平',
 '76': '平',
 '77': '仄',
 '78': '仄',
 '79': '平',
 '80': '仄',
 '81': '仄',
 '82': '仄',
 '83': '仄',
 '